# Yandex.Music

Comparison of Moscow and St. Petersburg is surrounded by myths. For example:
 * Moscow is a metropolis subject to the rigid rhythm of the working week;
 * St. Petersburg is a cultural capital, with its own tastes.

Using Yandex.Music data, you will compare the behavior of users in the two capitals.

**Research Purpose** - Test three hypotheses:
1. User activity depends on the day of the week. Moreover, in Moscow and St. Petersburg this manifests itself in different ways.
2. On Monday morning, certain genres prevail in Moscow, while others prevail in St. Petersburg. Similarly, Friday evenings are dominated by different genres, depending on the city.
3. Moscow and St. Petersburg prefer different genres of music. In Moscow, they listen to pop music more often, in St. Petersburg - Russian rap.

**Research Progress**

You will receive data on user behavior from the `yandex_music_project.csv` file. Nothing is known about the quality of the data. Therefore, before testing hypotheses, a review of the data is needed.

You will check the data for errors and assess their impact on the study. Then, during the pre-processing phase, you will look for opportunities to correct the most critical data errors.
 
Thus, the study will take place in three stages:
 1. Data review.
 2. Data preprocessing.
 3. Hypothesis testing.


## Browse data

Get the first idea about Yandex.Music data.

The main analytics tool is `pandas`. Import this library.

In [1]:
import pandas as pd

Read the `yandex_music_project.csv` file from the `/datasets` folder and save it in the `df` variable:

In [2]:
df = pd.read_csv('/datasets/yandex_music_project.csv')

Display the first ten rows of the table:

In [3]:
df.head(10)

,userID,Track,artist,genre,City,time,Day
0,FFB692EC,Kamigata To Boots,The Mass Missile,rock,Saint-Petersburg,20:28:33,Wednesday
1,55204538,Delayed Because of Accident,Andreas Rönnberg,rock,Moscow,14:07:09,Friday
2,20EC38,Funiculì funiculà,Mario Lanza,pop,Saint-Petersburg,20:58:07,Wednesday
3,A3DD03C9,Dragons in the Sunset,Fire + Ice,folk,Saint-Petersburg,08:37:09,Monday
4,E2DC1FAE,Soul People,Space Echo,dance,Moscow,08:34:34,Monday
5,842029A1,Преданная,IMPERVTOR,rusrap,Saint-Petersburg,13:09:41,Friday
6,4CB90AA5,True,Roman Messer,dance,Moscow,13:00:07,Wednesday
7,F03E1C1F,Feeling This Way,Polina Griffith,dance,Moscow,20:47:49,Wednesday
8,8FA1D3BE,И вновь продолжается бой,NaN,ruspop,Moscow,09:17:40,Friday
9,E772D5C0,Pessimist,NaN,dance,Saint-Petersburg,21:20:49,Wednesday


One command to get general information about the table:

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 65079 entries, 0 to 65078
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0     userID  65079 non-null  object
 1   Track     63848 non-null  object
 2   artist    57876 non-null  object
 3   genre     63881 non-null  object
 4     City    65079 non-null  object
 5   time      65079 non-null  object
 6   Day       65079 non-null  object
dtypes: object(7)
memory usage: 3.5+ MB


So the table has seven columns. The data type in all columns is `object`.

According to the data documentation:
* `userID` - user ID;
* `Track` — track name;
* `artist` — artist name;
* `genre` — genre name;
* `City` - user's city;
* `time` - start time of listening;
* `Day` is the day of the week.

There are three style violations in the column headings:
1. Lowercase letters are combined with uppercase.
2. There are gaps.

The number of values in the columns varies. This means there are missing values in the data.

**Conclusions**

Each line of the table contains data about the track you have listened to. Some of the columns describe the composition itself: title, artist and genre. The rest of the data tells about the user: what city he is from, when he listened to music.

Preliminarily, it can be argued that there is enough data to test hypotheses. But there are gaps in the data, and discrepancies in the names of the columns with good style.

To move forward, you need to fix problems in the data.

## Data preprocessing
Correct the style in the column headings, eliminate gaps. Then check the data for duplicates.

### Heading style
Display the column names:

In [5]:
df.columns # list of column names of table

Index(['  userID', 'Track', 'artist', 'genre', '  City  ', 'time', 'Day'], dtype='object')

Bring titles in line with good style:
* write a few words in the title in "snake_register",
* make all characters lowercase,
* eliminate spaces.

To do this, rename the columns like this:
* `'userID'` → `'user_id'`;
* ``Track'` → ``track'`;
* `'City'` → `'city'`;
* `'Day'` → `'day'`.

In [6]:
#renaming columns
df = df.rename(columns={'  userID': 'user_id', 'Track': 'track', '  City  ': 'city', 'Day': 'day'})

Check the result. To do this, display the column names again:

In [7]:
df.columns # check results - list column names

Index(['user_id', 'track', 'artist', 'genre', 'city', 'time', 'day'], dtype='object')

### Missing values
First, count how many missing values are in the table. Two `pandas` methods are sufficient for this:

In [11]:
df.isna().sum() # skip count

user_id       0
track      1231
artist     7203
genre      1198
city          0
time          0
day           0
dtype: int64

Not all missing values affect the study. So in `track` and `artist` the gaps are not important for your work. It suffices to replace them with explicit notation.

But omissions in `genre` can interfere with the comparison of musical tastes in Moscow and St. Petersburg. In practice, it would be correct to determine the cause of the gaps and restore the data. This option is not available in the curriculum. Have to:
* fill in these gaps with explicit notation,
* estimate how much they will damage the calculations.

Replace the missing values in the `track`, `artist` and `genre` columns with the string `'unknown'`. To do this, create a `columns_to_replace` list, iterate through its elements with a `for` loop, and for each column, replace the missing values:

In [12]:
# loop through column names and replace missing values with 'unknown'

columns_to_replace = ['track', 'artist', 'genre']

for column in columns_to_replace:
    df[column] = df[column].fillna('unknown')


Make sure there are no gaps in the table. To do this, count the missing values again.

In [13]:
df.isna().sum() # skip count

user_id    0
track      0
artist     0
genre      0
city       0
time       0
day        0
dtype: int64

### Duplicates
Count explicit duplicates in a table with one command:

In [14]:
df.duplicated().sum() # counting duplicates

3826

Call the special `pandas` method to remove obvious duplicates:

In [15]:
# removal of explicit duplicates (with the removal of old indexes and the formation of new ones)

df = df.drop_duplicates().reset_index(drop=True)

Once again, count the obvious duplicates in the table - make sure you get rid of them completely:

In [16]:
df.duplicated().sum() # check for duplicates

0

Now get rid of the implicit duplicates in the `genre` column. For example, the name of the same genre can be spelled slightly differently. Such errors will also affect the result of the study.

Display a list of unique genre names sorted alphabetically. For this:
* extract the desired dataframe column,
* apply a sort method to it,
* for a sorted column, call a method that will return the unique values from the column.

In [17]:
print('Number of unique genres before fixing implicit duplicates:', end=' ')
print(len(df['genre'].unique()))

df['genre'].sort_values().unique() # View Unique Genre Names

Количество уникальных жанров до исправления неявных дубликатов: 290


array(['acid', 'acoustic', 'action', 'adult', 'africa', 'afrikaans',
       'alternative', 'alternativepunk', 'ambient', 'americana',
       'animated', 'anime', 'arabesk', 'arabic', 'arena',
       'argentinetango', 'art', 'audiobook', 'author', 'avantgarde',
       'axé', 'baile', 'balkan', 'beats', 'bigroom', 'black', 'bluegrass',
       'blues', 'bollywood', 'bossa', 'brazilian', 'breakbeat', 'breaks',
       'broadway', 'cantautori', 'cantopop', 'canzone', 'caribbean',
       'caucasian', 'celtic', 'chamber', 'chanson', 'children', 'chill',
       'chinese', 'choral', 'christian', 'christmas', 'classical',
       'classicmetal', 'club', 'colombian', 'comedy', 'conjazz',
       'contemporary', 'country', 'cuban', 'dance', 'dancehall',
       'dancepop', 'dark', 'death', 'deep', 'deutschrock', 'deutschspr',
       'dirty', 'disco', 'dnb', 'documentary', 'downbeat', 'downtempo',
       'drum', 'dub', 'dubstep', 'eastern', 'easy', 'electronic',
       'electropop', 'emo', 'entehno', '

Look through the list and look for implicit duplicates of the name `hiphop`. These may be misspelled titles or alternative titles in the same genre.

You will see the following implicit duplicates:
**hip*,
**hop*,
**hip-hop*

And also the values:

* *'electronic'*
* *'electronics'*
**'poi'* - this value most likely arose due to incorrect encoding.


To clear the table of them, write the `replace_wrong_genres()` function with two parameters:
* `wrong_genres` - list of duplicates,
* `correct_genre` is a string with the correct value.

The function should correct the `genre` column in the `df` table: replace each value from the `wrong_genres` list with a value from `correct_genre`.

In [18]:
# Function to replace implicit duplicates
def replace_wrong_genres(wrong_genres, correct_genre):
    df['genre'] = df['genre'].replace(wrong_genres, correct_genre)

Call `replace_wrong_genres()` and pass it arguments such that it eliminates implicit duplicates: instead of `hip`, `hop` and `hip-hop` the table should have the value `hiphop`:

In [19]:
# Eliminate Implicit Duplicates
wrong_genres_hiphop = ['hip', 'hop', 'hip-hop' ]

for value in wrong_genres_hiphop:
    replace_wrong_genres(value, 'hiphop')

In [20]:
replace_wrong_genres('электроника', 'electronic')
replace_wrong_genres('ïîï', 'unknown')

Check that you have replaced the wrong names:

* hip
* hop
* hip-hop

Print a sorted list of unique values in the `genre` column:

In [21]:
print('Number of unique genres after fixing implicit duplicates:', end=' ')
print(len(df['genre'].unique()))

df['genre'].sort_values().unique() # Checking for Implicit Duplicates

Количество уникальных жанров после исправления неявных дубликатов: 285


array(['acid', 'acoustic', 'action', 'adult', 'africa', 'afrikaans',
       'alternative', 'alternativepunk', 'ambient', 'americana',
       'animated', 'anime', 'arabesk', 'arabic', 'arena',
       'argentinetango', 'art', 'audiobook', 'author', 'avantgarde',
       'axé', 'baile', 'balkan', 'beats', 'bigroom', 'black', 'bluegrass',
       'blues', 'bollywood', 'bossa', 'brazilian', 'breakbeat', 'breaks',
       'broadway', 'cantautori', 'cantopop', 'canzone', 'caribbean',
       'caucasian', 'celtic', 'chamber', 'chanson', 'children', 'chill',
       'chinese', 'choral', 'christian', 'christmas', 'classical',
       'classicmetal', 'club', 'colombian', 'comedy', 'conjazz',
       'contemporary', 'country', 'cuban', 'dance', 'dancehall',
       'dancepop', 'dark', 'death', 'deep', 'deutschrock', 'deutschspr',
       'dirty', 'disco', 'dnb', 'documentary', 'downbeat', 'downtempo',
       'drum', 'dub', 'dubstep', 'eastern', 'easy', 'electronic',
       'electropop', 'emo', 'entehno', '

**Conclusions**

Preprocessing found three problems in the data:

- headline style violations,
- missing values,
- duplicates - explicit and implicit.

You've fixed the headers to make the table easier to work with. Without duplicates, the study will become more accurate.

You have replaced missing values with `'unknown'`. It remains to be seen whether the gaps in the `genre` column will harm the study.

Now we can move on to hypothesis testing.

## Testing hypotheses

### Comparison of user behavior in two cities

The first hypothesis states that users listen to music differently in Moscow and St. Petersburg. Check this assumption against the data on the three days of the week - Monday, Wednesday and Friday. For this:

* Separate Moscow and St. Petersburg users
* Compare how many tracks each user group listened to on Monday, Wednesday and Friday.

For training, first perform each of the calculations separately.

Estimate user activity in each city. Group the data by city and count the plays in each group.

In [22]:
df['city'].unique() #checking cities

array(['Saint-Petersburg', 'Moscow'], dtype=object)

In [23]:
# Counting plays in each city
print('Общее количество прослушиваний в Москве:', df[df['city']=='Moscow']['track'].count())
print('Общее количество прослушиваний в Санкт-Петербурге:', df[df['city']=='Saint-Petersburg']['track'].count())

Общее количество прослушиваний в Москве: 42741
Общее количество прослушиваний в Санкт-Петербурге: 18512


There are more auditions in Moscow than in St. Petersburg. It does not follow from this that Moscow users listen to music more often. There are simply more users in Moscow.

Now group the data by day of the week and count the plays on Monday, Wednesday, and Friday. Please note that the data only contains information about the plays for these days only.

In [24]:
df.groupby('day')['track'].count() # Play count for each of the three days

day
Friday       21840
Monday       21354
Wednesday    18059
Name: track, dtype: int64

On average, users from the two cities are less active on Wednesdays. But the picture may change if we consider each city separately.

You have seen how grouping by city and by day of the week works. Now write a function that combines these two calculations.

Create a `number_tracks()` function that will count the plays for a given day and city. She needs two parameters:
* day of the week,
* city name.

In the function, save to a variable the rows of the source table that have the value:
  * in the `day` column is equal to the `day` parameter,
  * in the `city` column is equal to the `city` parameter.

To do this, apply sequential filtering with logical indexing.

Then count the values ​​in the `user_id` column of the resulting table. Save the result to a new variable. Return this variable from the function.

In [25]:
# <create number_tracks() function>
# A function is declared with two parameters: day, city.
# The track_list variable stores those rows of the df table for which
# the value in the 'day' column is equal to the day parameter and at the same time the value
# in the 'city' column is equal to the city parameter (use sequential filtering
# using logical indexing).
# The track_list_count variable stores the number of values in the 'user_id' column,
# calculated by the count() method for the track_list table.
# The function returns a number - the value of track_list_count.

# Function for counting plays for a specific city and day.
# Using sequential filtering with logical indexing, it
# first get the rows with the desired day from the source table,
# then from the result it will filter the rows with the desired city,
# the count() method will count the number of values in the user_id column.
# This is the number the function will return as a result

def number_tracks(day, city):
    track_list = df[(df['day']==day) & (df['city']==city)]
    track_list_count = track_list['user_id'].count()
    return track_list_count

Call `number_tracks()` six times, changing the parameters so that you get data for each city on each of the three days.

In [26]:
# number of auditions in Moscow on Mondays
number_tracks('Monday', 'Moscow')

15740

In [27]:
# number of auditions in St. Petersburg on Mondays
number_tracks('Monday', 'Saint-Petersburg')

5614

In [28]:
# number of auditions in Moscow on Wednesdays
number_tracks('Wednesday', 'Moscow')

11056

In [29]:
# number of auditions in St. Petersburg on Wednesdays
number_tracks('Wednesday', 'Saint-Petersburg')

7003

In [30]:
# number of auditions in Moscow on Fridays
number_tracks('Friday', 'Moscow')

15945

In [31]:
# number of auditions in St. Petersburg on Fridays
number_tracks('Friday', 'Saint-Petersburg')

5895

Create a table using the `pd.DataFrame` constructor, where
* column names - `['city', 'monday', 'wednesday', 'friday']`;
* data is the results you got with `number_tracks`.

In [32]:
# Results table

first_hypothesis_columns = ['city', 'monday', 'wednesday', 'friday'] #column names
first_hypothesis_data=[]                                             #list with data

for city in ['Moscow', 'Saint-Petersburg']:                          #creating a list with data
    result_city = []
    result_city.append(city)
    for day in ['Monday', 'Wednesday', 'Friday']:
        result_city.append(number_tracks(day, city))
    first_hypothesis_data.append(result_city)

first_hypothesis_df = pd.DataFrame(columns=first_hypothesis_columns, data=first_hypothesis_data)
display(first_hypothesis_df)

,city,monday,wednesday,friday
0,Moscow,15740,11056,15945
1,Saint-Petersburg,5614,7003,5895


**Conclusions**

The data shows the difference in user behavior:

- In Moscow, the peak of listening falls on Monday and Friday, and on Wednesday there is a noticeable decline.
- In St. Petersburg, on the contrary, they listen to music more on Wednesdays. Activity on Monday and Friday here is almost equally inferior to Wednesday.

So the data support the first hypothesis.

### Music at the beginning and end of the week

According to the second hypothesis, on Monday morning certain genres predominate in Moscow, while others dominate in St. Petersburg. Similarly, Friday evenings are dominated by different genres, depending on the city.

Save tables with data in two variables:
* in Moscow - in `moscow_general`;
* in St. Petersburg - in `spb_general`.

In [33]:
# getting the moscow_general table from those rows of the df table,
# for which the value in the 'city' column is 'Moscow'
moscow_general = df[df['city']=='Moscow']

In [34]:
# getting the spb_general table from those rows of the df table,
# for which the value in the 'city' column is 'Saint-Petersburg'
spb_general = df[df['city']=='Saint-Petersburg']

Create a function `genre_weekday()` with four parameters:
* table (dataframe) with data,
* day of the week,
* start timestamp in 'hh:mm' format,
* last timestamp in 'hh:mm' format.

The function should return information about the top 10 genres of those tracks that were listened to on the specified day, in the interval between two timestamps.

In [35]:
# Declaring the function genre_weekday() with parameters table, day, time1, time2,
# which returns information about the most popular genres on the specified day in
# given time:
# 1) the genre_df variable saves those rows of the transmitted dataframe table, for
# of which at the same time:
# - the value in the day column is equal to the value of the day argument
# - the value in the time column is greater than the value of the time1 argument
# - the value in the time column is less than the value of the time2 argument
# Use sequential filtering with boolean indexing.
# 2) group dataframe genre_df by genre column, take one of its
# columns and use the count() method to count the number of entries for each
# of present genres, write the resulting Series to a variable
#genre_df_count
# 3) sort genre_df_count in descending order of occurrence and save
# into the genre_df_sorted variable
# 4) return a Series of the first 10 genre_df_sorted values, these will be the top 10
# popular genres (on the specified day, at the specified time)

def genre_weekday(table, day, time1, time2):
    genre_df = table[table['day']==day]
    genre_df = genre_df[(genre_df['time']>time1) & (genre_df['time']<time2)]
    genre_df_count = genre_df.groupby('genre')['track'].count()
    genre_df_sorted = genre_df_count.sort_values(ascending=False)
    return genre_df_sorted.head(10)

Compare the results of the `genre_weekday()` function for Moscow and St. Petersburg on Monday morning (from 7:00 to 11:00) and Friday evening (from 17:00 to 23:00):

In [36]:
# function call for Monday morning in Moscow (instead of df — moscow_general table)
# time objects are strings and are compared as strings
# call example: genre_weekday(moscow_general, 'Monday', '07:00', '11:00')
genre_weekday(moscow_general, 'Monday', '07:00', '11:00')

genre
pop            781
dance          549
electronic     480
rock           474
hiphop         286
ruspop         186
world          181
rusrap         175
alternative    164
unknown        161
Name: track, dtype: int64

In [37]:
# function call for Monday morning in St. Petersburg (instead of df - spb_general table)
genre_weekday(spb_general, 'Monday', '07:00', '11:00')

genre
pop            218
dance          182
rock           162
electronic     147
hiphop          80
ruspop          64
alternative     58
rusrap          55
jazz            44
classical       40
Name: track, dtype: int64

In [38]:
# function call for Friday evening in Moscow
genre_weekday(moscow_general, 'Friday', '17:00', '23:00')

genre
pop            713
rock           517
dance          495
electronic     482
hiphop         273
world          208
ruspop         170
alternative    163
classical      163
rusrap         142
Name: track, dtype: int64

In [39]:
# function call for Friday evening in St. Petersburg
genre_weekday(spb_general, 'Friday', '17:00', '23:00')

genre
pop            256
electronic     216
rock           216
dance          210
hiphop          97
alternative     63
jazz            61
classical       60
rusrap          59
world           54
Name: track, dtype: int64

**Conclusions**

If we compare the top 10 genres on Monday morning, we can draw the following conclusions:

1. In Moscow and St. Petersburg they listen to similar music. The only difference is that the Moscow rating includes the “world” genre, while the St. Petersburg rating includes jazz and classical.

2. There were so many missing values ​​in Moscow that the value `'unknown'` took tenth place among the most popular genres. This means that missing values ​​occupy a significant share in the data and threaten the reliability of the study.

Friday night does not change this picture. Some genres rise a little higher, others go down, but overall the top 10 stays the same.

Thus, the second hypothesis was only partially confirmed:
* Users listen to similar music at the beginning of the week and at the end.
* The difference between Moscow and St. Petersburg is not very pronounced. In Moscow, they listen to Russian popular music more often, in St. Petersburg - jazz.

However, gaps in the data cast doubt on this result. There are so many of them in Moscow that the top 10 ranking could look different if it were not for the lost genre data.

### Genre preferences in Moscow and St. Petersburg

Hypothesis: St. Petersburg is the capital of rap, the music of this genre is listened to more often than in Moscow. And Moscow is a city of contrasts, which, nevertheless, is dominated by pop music.

Group the `moscow_general` table by genre and count the plays of each genre's tracks using the `count()` method. Then sort the result in descending order and store it in the `moscow_genres` table.

In [40]:
# in one line: grouping the moscow_general table by the 'genre' column,
# counting the number of 'genre' values in this grouping using the count() method,
# sort the resulting Series in descending order and save to moscow_genres
moscow_genres = moscow_general.groupby('genre')['genre'].count().sort_values(ascending=False)

Print the first ten lines of `moscow_genres` to the screen:

In [41]:
# view the first 10 lines of moscow_genres
moscow_genres.head(10)

genre
pop            5892
dance          4435
rock           3965
electronic     3786
hiphop         2096
classical      1616
world          1432
alternative    1379
ruspop         1372
rusrap         1161
Name: genre, dtype: int64

Now repeat the same for Petersburg.

Group the `spb_general` table by genre. Count the number of listens to tracks of each genre. Sort the result in descending order and store it in the `spb_genres` table:

In [42]:
# in one line: group spb_general table by 'genre' column,
# counting the number of 'genre' values in this grouping using the count() method,
# sort the resulting Series in descending order and store it in spb_genres
spb_genres = spb_general.groupby('genre')['genre'].count().sort_values(ascending=False)

Print the first ten lines of `spb_genres` to the screen:

In [43]:
# view the first 10 lines of spb_genres
spb_genres.head(10)

genre
pop            2431
dance          1932
rock           1879
electronic     1737
hiphop          960
alternative     649
classical       646
rusrap          564
ruspop          538
world           515
Name: genre, dtype: int64

**Conclusions**


The hypothesis was partially confirmed:
* Pop music is the most popular genre in Moscow, as the hypothesis suggested. Moreover, in the top 10 genres there is a close genre - Russian popular music.
* Contrary to expectations, rap is equally popular in Moscow and St. Petersburg.

## Results of the study

You tested three hypotheses and found:

1. The day of the week has a different effect on the activity of users in Moscow and St. Petersburg.

The first hypothesis was fully confirmed.

2. Musical preferences do not change much during the week - be it Moscow or St. Petersburg. Small differences are noticeable at the beginning of the week, on Mondays:
* in Moscow they listen to music of the “world” genre,
* in St. Petersburg - jazz and classical music.

Thus, the second hypothesis was only partly confirmed. This result could have been different were it not for gaps in the data.

3. The tastes of users of Moscow and St. Petersburg have more in common than differences. Contrary to expectations, genre preferences in St. Petersburg resemble those in Moscow.

The third hypothesis was not confirmed. If there are differences in preferences, they are invisible to the bulk of users.

**In practice, studies contain tests of statistical hypotheses.**
From the data of one service, it is not always possible to draw a conclusion about all the inhabitants of the city.
Tests of statistical hypotheses will show how reliable they are, based on the available data.
You will learn more about hypothesis testing methods in the following topics.